# Imports

Q1: Entity schedule has "Port Aventura World" in it

In [2]:
import polars as pl
from polars import col
import polars.selectors as cs
import pyarrow as pa
import pandas as pd
import numpy as np

# Set the plotting backend to Plotly
pd.options.plotting.backend = "plotly"

from datetime import date

# Attendance

In [3]:
attendance = (
  pl.read_csv('data/attendance.csv').filter(col('FACILITY_NAME') == 'PortAventura World')
  .with_columns(col('USAGE_DATE').str.to_datetime("%m/%d/%Y"))
  .select('USAGE_DATE', 'attendance')
)

In [4]:
attendance.head()

USAGE_DATE,attendance
datetime[μs],i64
2018-06-01 00:00:00,46804
2018-06-02 00:00:00,57940
2018-06-03 00:00:00,44365
2018-06-04 00:00:00,37617
2018-06-05 00:00:00,32438


In [5]:
attendance.to_pandas().plot(x= 'USAGE_DATE', y='attendance')

In [6]:
attendance.filter(
  (col('USAGE_DATE') <= date(2020, 3, 13)).or_(col('USAGE_DATE') >= date(2021, 6, 19))
).groupby(col('USAGE_DATE').dt.weekday()).mean().sort('USAGE_DATE', descending=False)

C:\Users\cedom\AppData\Local\Temp\ipykernel_21492\751617668.py:3: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.



USAGE_DATE,attendance
i8,f64
1,35889.966887
2,35179.397351
3,34161.306667
4,35441.28
5,38461.589404
6,49241.701987
7,40890.675497


In [7]:
attendance.filter(
  (col('USAGE_DATE') <= date(2020, 3, 13)).or_(col('USAGE_DATE') >= date(2021, 6, 19))
).groupby(col('USAGE_DATE').dt.weekday()).mean().sort('USAGE_DATE', descending=False).to_pandas().plot(x = 'USAGE_DATE', y= 'attendance')

C:\Users\cedom\AppData\Local\Temp\ipykernel_21492\839305555.py:3: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.



# Link attraction-park

In [8]:
link = pl.read_csv('data/link_attraction_park.csv', separator=';')

# Entity Schedule

In [9]:
link.head()

ATTRACTION,PARK
str,str
"""Aeroplane Ride…","""Tivoli Gardens…"
"""Bumper Cars""","""PortAventura W…"
"""Bungee Jump""","""PortAventura W…"
"""Circus Train""","""PortAventura W…"
"""Crazy Bus""","""Tivoli Gardens…"


In [10]:
schedule = (
  pl.read_csv('data/entity_schedule.csv')
  .join(link, left_on = 'ENTITY_DESCRIPTION_SHORT', right_on = 'ATTRACTION', how='left')
  .filter(col('PARK') == 'PortAventura World')
  .with_columns(cs.contains(('TIME', 'DATE')).str.to_datetime())
)

In [11]:
schedule.head()

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
null,"""Dizzy Dropper""","""ATTR""",2022-04-07 08:30:00,2022-04-07 22:04:00,2022-04-08 08:00:30,2022-04-07 00:00:00,"""PortAventura W…"
"""Fermeture Réha…","""Kiddie Coaster…","""ATTR""",2018-02-21 23:59:00,2018-02-21 23:59:00,2018-02-22 08:36:23,2018-02-21 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-09-04 23:59:00,2018-09-04 23:59:00,2018-09-05 08:12:54,2018-09-04 00:00:00,"""PortAventura W…"
null,"""Free Fall""","""ATTR""",2020-01-04 08:11:00,2020-01-04 18:15:00,2020-01-05 07:52:42,2020-01-04 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-06-24 23:59:00,2018-06-24 23:59:00,2018-06-25 08:03:37,2018-06-24 00:00:00,"""PortAventura W…"


In [12]:
schedule.filter(col('REF_CLOSING_DESCRIPTION')== 'Fermeture Opérationnelle').sort('ENTITY_DESCRIPTION_SHORT', 'DEB_TIME', descending=[False, False])

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-27 23:59:00,2019-07-27 23:59:00,2019-07-28 08:08:32,2019-07-27 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-28 23:59:00,2019-07-28 23:59:00,2019-07-29 07:54:10,2019-07-28 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-29 23:59:00,2019-07-29 23:59:00,2019-07-30 07:37:39,2019-07-29 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-30 23:59:00,2019-07-30 23:59:00,2019-07-31 07:37:04,2019-07-30 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-31 23:59:00,2019-07-31 23:59:00,2019-08-01 07:54:33,2019-07-31 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-08-01 23:59:00,2019-08-01 23:59:00,2019-08-02 08:08:55,2019-08-01 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Go-Karts""","""ATTR""",2022-05-30 23:59:00,2022-05-30 23:59:00,2022-05-31 08:16:56,2022-05-30 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Himalaya Ride""","""ATTR""",2019-07-15 01:00:00,2019-07-15 01:00:00,2019-07-16 07:41:56,2019-07-15 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Himalaya Ride""","""ATTR""",2019-07-17 23:59:00,2019-07-17 23:59:00,2019-07-18 07:39:24,2019-07-17 00:00:00,"""PortAventura W…"


# Waiting times

In [13]:
wait = (
  pl.read_csv('data/waiting_times.csv')
  .join(link, left_on = 'ENTITY_DESCRIPTION_SHORT', right_on = 'ATTRACTION', how='left')
  .filter(col('PARK') == 'PortAventura World')
)

In [14]:
wt = (
  wait
  .with_columns(
    col('DEB_TIME').str.to_datetime(),
    col('FIN_TIME').str.to_datetime(),
  )
)

In [15]:
wt.head()

WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,PARK
str,datetime[μs],i64,datetime[μs],str,i64,f64,f64,f64,f64,i64,i64,i64,f64,str
"""2018-01-01""",2018-01-01 21:00:00,21,2018-01-01 21:15:00,"""Roller Coaster…",0,2.0,0.0,0.0,0.0,0,0,0,2.0,"""PortAventura W…"
"""2018-01-01""",2018-01-01 19:30:00,19,2018-01-01 19:45:00,"""Bumper Cars""",5,18.0,148.0,254.749,254.75,15,15,0,18.0,"""PortAventura W…"
"""2018-01-01""",2018-01-01 22:30:00,22,2018-01-01 22:45:00,"""Rapids Ride""",0,1.0,0.0,0.0,0.0,0,0,0,2.0,"""PortAventura W…"
"""2018-01-01""",2018-01-01 12:45:00,12,2018-01-01 13:00:00,"""Crazy Dance""",5,1.0,46.0,250.001,250.0,15,15,0,1.0,"""PortAventura W…"
"""2018-01-01""",2018-01-01 18:15:00,18,2018-01-01 18:30:00,"""Free Fall""",50,3.0,0.0,0.0,0.0,0,0,0,3.0,"""PortAventura W…"


In [16]:
# wt = (
#   wait
#   .with_columns(
#     pl.col("WORK_DATE").str.strptime(pl.Date, "%m/%d/%Y").cast(pl.Datetime).alias("WORK_DATE_DT"))
#   .with_columns(
#     (pl.col("WORK_DATE_DT") + pl.duration(hours = "DEB_TIME_HOUR") + pl.duration(minutes = col("DEB_TIME").str.split(":").list.get(0).cast(pl.Int32))).alias("DEB_DATETIME"),
#     (pl.col("WORK_DATE_DT") + pl.duration(hours = "DEB_TIME_HOUR") + pl.duration(minutes = col("FIN_TIME").str.split(":").list.get(0).cast(pl.Int32))).alias("FIN_DATETIME"),
#     )
#   .drop('WORK_DATE_DT', 'WORK_DATE', 'DEB_TIME', 'FIN_TIME', 'DEB_TIME_HOUR')
# )

In [17]:
top_6_rides = (
  wt
  .group_by(col('ENTITY_DESCRIPTION_SHORT'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
).sort('WAIT_TIME_MAX_mean').tail(6).to_numpy()

[sublist[0] for sublist in top_6_rides]

C:\Users\cedom\AppData\Local\Temp\ipykernel_21492\3737640886.py:4: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



['Spinning Coaster',
 'Go-Karts',
 'Free Fall',
 'Swing Ride',
 'Giant Wheel',
 'Spiral Slide']

In [18]:
(
  wt
  .filter((col('DEB_TIME') <= date(2020, 3, 13)).or_(col('DEB_TIME') >= date(2021, 6, 19)))   # We take out COVID TIME
  .group_by(col('ENTITY_DESCRIPTION_SHORT').alias('RIDE_NAME'), col('DEB_TIME').dt.weekday().alias('weekday'), (col('DEB_TIME').dt.hour() + col('DEB_TIME').dt.minute() / 60).alias('hour_minute'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
  .filter(col('RIDE_NAME').is_in(['Spinning Coaster', 'Go-Karts', 'Free Fall', 'Swing Ride', 'Giant Wheel', 'Spiral Slide']))
).sort(['weekday', 'hour_minute'], descending=[False, False]).to_pandas().plot(
  x= 'hour_minute',
  y='WAIT_TIME_MAX_mean',
  color='weekday',
  facet_col="RIDE_NAME",
  facet_col_wrap = 3,
  title = 'Average wait time (m) at attraction X throughout the day', width=2000, height=1000) 

C:\Users\cedom\AppData\Local\Temp\ipykernel_21492\2762528156.py:5: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



In [19]:
(
  wt
  .filter((col('DEB_TIME') <= date(2020, 3, 13)).or_(col('DEB_TIME') >= date(2021, 6, 19)))   # We take out COVID TIME
  .group_by(col('ENTITY_DESCRIPTION_SHORT').alias('RIDE_NAME'), col('DEB_TIME').dt.weekday().alias('weekday'), (col('DEB_TIME').dt.hour() + col('DEB_TIME').dt.minute() / 60).alias('hour_minute'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
  .filter(col('RIDE_NAME').is_in(['Spinning Coaster', 'Go-Karts', 'Free Fall', 'Swing Ride', 'Giant Wheel', 'Spiral Slide']))
).sort(['weekday', 'hour_minute'], descending=[False, False]).to_pandas()

C:\Users\cedom\AppData\Local\Temp\ipykernel_21492\3230232856.py:5: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



,RIDE_NAME,weekday,hour_minute,WAIT_TIME_MAX_mean
0,Go-Karts,1,9.00,7.784091
1,Swing Ride,1,9.00,19.261364
2,Free Fall,1,9.00,4.346591
3,Spinning Coaster,1,9.00,3.857143
4,Giant Wheel,1,9.00,8.948864
...,...,...,...,...
2347,Free Fall,7,22.75,0.000000
2348,Spiral Slide,7,22.75,1.542857
2349,Giant Wheel,7,22.75,5.000000
2350,Swing Ride,7,22.75,0.714286


# Full Table

In [20]:
schedule

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
null,"""Dizzy Dropper""","""ATTR""",2022-04-07 08:30:00,2022-04-07 22:04:00,2022-04-08 08:00:30,2022-04-07 00:00:00,"""PortAventura W…"
"""Fermeture Réha…","""Kiddie Coaster…","""ATTR""",2018-02-21 23:59:00,2018-02-21 23:59:00,2018-02-22 08:36:23,2018-02-21 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-09-04 23:59:00,2018-09-04 23:59:00,2018-09-05 08:12:54,2018-09-04 00:00:00,"""PortAventura W…"
null,"""Free Fall""","""ATTR""",2020-01-04 08:11:00,2020-01-04 18:15:00,2020-01-05 07:52:42,2020-01-04 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-06-24 23:59:00,2018-06-24 23:59:00,2018-06-25 08:03:37,2018-06-24 00:00:00,"""PortAventura W…"
null,"""Giga Coaster""","""ATTR""",2019-04-08 09:25:00,2019-04-08 21:00:00,2019-04-09 07:45:24,2019-04-08 00:00:00,"""PortAventura W…"
null,"""Inverted Coast…","""ATTR""",2018-09-15 09:55:00,2018-09-15 18:20:00,2018-09-16 08:38:49,2018-09-15 00:00:00,"""PortAventura W…"
null,"""Himalaya Ride""","""ATTR""",2018-06-05 10:00:00,2018-06-05 23:00:00,2018-06-06 07:27:32,2018-06-05 00:00:00,"""PortAventura W…"
null,"""Water Ride""","""ATTR""",2019-05-02 08:30:00,2019-05-02 21:14:00,2019-06-05 09:12:18,2019-05-02 00:00:00,"""PortAventura W…"


In [21]:
# We have no data on attendance pre 2018-06-01, so we will drop all waiting time rows from before that

wtf = (
  wt.sort('DEB_TIME', descending=False).set_sorted(column='DEB_TIME')
  .join_asof(attendance.set_sorted(column='USAGE_DATE'), left_on='DEB_TIME', right_on='USAGE_DATE')
  .filter(col('USAGE_DATE').is_not_null())
  .drop('WORK_DATE')
)

In [22]:
schedule

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
null,"""Dizzy Dropper""","""ATTR""",2022-04-07 08:30:00,2022-04-07 22:04:00,2022-04-08 08:00:30,2022-04-07 00:00:00,"""PortAventura W…"
"""Fermeture Réha…","""Kiddie Coaster…","""ATTR""",2018-02-21 23:59:00,2018-02-21 23:59:00,2018-02-22 08:36:23,2018-02-21 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-09-04 23:59:00,2018-09-04 23:59:00,2018-09-05 08:12:54,2018-09-04 00:00:00,"""PortAventura W…"
null,"""Free Fall""","""ATTR""",2020-01-04 08:11:00,2020-01-04 18:15:00,2020-01-05 07:52:42,2020-01-04 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-06-24 23:59:00,2018-06-24 23:59:00,2018-06-25 08:03:37,2018-06-24 00:00:00,"""PortAventura W…"
null,"""Giga Coaster""","""ATTR""",2019-04-08 09:25:00,2019-04-08 21:00:00,2019-04-09 07:45:24,2019-04-08 00:00:00,"""PortAventura W…"
null,"""Inverted Coast…","""ATTR""",2018-09-15 09:55:00,2018-09-15 18:20:00,2018-09-16 08:38:49,2018-09-15 00:00:00,"""PortAventura W…"
null,"""Himalaya Ride""","""ATTR""",2018-06-05 10:00:00,2018-06-05 23:00:00,2018-06-06 07:27:32,2018-06-05 00:00:00,"""PortAventura W…"
null,"""Water Ride""","""ATTR""",2019-05-02 08:30:00,2019-05-02 21:14:00,2019-06-05 09:12:18,2019-05-02 00:00:00,"""PortAventura W…"


In [23]:
schedule.filter(col('REF_CLOSING_DESCRIPTION')== 'Fermeture Opérationnelle').sort('ENTITY_DESCRIPTION_SHORT', 'DEB_TIME', descending=[False, False])

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-27 23:59:00,2019-07-27 23:59:00,2019-07-28 08:08:32,2019-07-27 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-28 23:59:00,2019-07-28 23:59:00,2019-07-29 07:54:10,2019-07-28 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-29 23:59:00,2019-07-29 23:59:00,2019-07-30 07:37:39,2019-07-29 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-30 23:59:00,2019-07-30 23:59:00,2019-07-31 07:37:04,2019-07-30 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-07-31 23:59:00,2019-07-31 23:59:00,2019-08-01 07:54:33,2019-07-31 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Drop Tower""","""ATTR""",2019-08-01 23:59:00,2019-08-01 23:59:00,2019-08-02 08:08:55,2019-08-01 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Go-Karts""","""ATTR""",2022-05-30 23:59:00,2022-05-30 23:59:00,2022-05-31 08:16:56,2022-05-30 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Himalaya Ride""","""ATTR""",2019-07-15 01:00:00,2019-07-15 01:00:00,2019-07-16 07:41:56,2019-07-15 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Himalaya Ride""","""ATTR""",2019-07-17 23:59:00,2019-07-17 23:59:00,2019-07-18 07:39:24,2019-07-17 00:00:00,"""PortAventura W…"


In [44]:
schedule.filter(col('REF_CLOSING_DESCRIPTION')== 'Fermeture Opérationnelle').filter(col('ENTITY_DESCRIPTION_SHORT') == "Go-Karts").sort('ENTITY_DESCRIPTION_SHORT', 'DEB_TIME', descending=[False, False])

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
"""Fermeture Opér…","""Go-Karts""","""ATTR""",2022-05-30 23:59:00,2022-05-30 23:59:00,2022-05-31 08:16:56,2022-05-30 00:00:00,"""PortAventura W…"


In [24]:
wtf.head()

DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,PARK,USAGE_DATE,attendance
datetime[μs],i64,datetime[μs],str,i64,f64,f64,f64,f64,i64,i64,i64,f64,str,datetime[μs],i64
2018-06-01 09:00:00,9,2018-06-01 09:15:00,"""Crazy Dance""",0,1.0,0.0,0.0,0.0,0,0,0,1.0,"""PortAventura W…",2018-06-01 00:00:00,46804
2018-06-01 09:00:00,9,2018-06-01 09:15:00,"""Circus Train""",0,1.0,0.0,0.0,0.0,0,0,0,1.0,"""PortAventura W…",2018-06-01 00:00:00,46804
2018-06-01 09:00:00,9,2018-06-01 09:15:00,"""Spiral Slide""",90,2.0,29.0,75.0,75.0,15,15,0,2.0,"""PortAventura W…",2018-06-01 00:00:00,46804
2018-06-01 09:00:00,9,2018-06-01 09:15:00,"""Inverted Coast…",0,1.0,0.0,0.0,0.0,0,0,0,3.0,"""PortAventura W…",2018-06-01 00:00:00,46804
2018-06-01 09:00:00,9,2018-06-01 09:15:00,"""Drop Tower""",5,16.0,100.0,140.25,140.25,15,15,0,16.0,"""PortAventura W…",2018-06-01 00:00:00,46804


In [43]:
wtf.filter(col('DEB_TIME') > date(2022, 5, 30)).filter(col('DEB_TIME') < date(2022, 6, 15)).filter(col('ENTITY_DESCRIPTION_SHORT') == "Go-Karts").filter(col('WAIT_TIME_MAX') > 0)

DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,PARK,USAGE_DATE,attendance
datetime[μs],i64,datetime[μs],str,i64,f64,f64,f64,f64,i64,i64,i64,f64,str,datetime[μs],i64
2022-06-11 09:00:00,9,2022-06-11 09:15:00,"""Go-Karts""",30,2.0,205.0,450.5,225.25,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 09:15:00,9,2022-06-11 09:30:00,"""Go-Karts""",40,2.0,208.0,450.5,225.25,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 09:30:00,9,2022-06-11 09:45:00,"""Go-Karts""",40,2.4,192.0,450.5,270.3,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 09:45:00,9,2022-06-11 10:00:00,"""Go-Karts""",50,4.0,187.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 10:00:00,10,2022-06-11 10:15:00,"""Go-Karts""",50,4.0,319.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 10:15:00,10,2022-06-11 10:30:00,"""Go-Karts""",50,4.0,323.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 10:30:00,10,2022-06-11 10:45:00,"""Go-Karts""",45,4.0,325.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 10:45:00,10,2022-06-11 11:00:00,"""Go-Karts""",45,4.0,288.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184
2022-06-11 11:00:00,11,2022-06-11 11:15:00,"""Go-Karts""",55,4.0,242.0,450.5,450.5,15,15,0,4.0,"""PortAventura W…",2022-06-11 00:00:00,53184


In [47]:
wtf.filter(col('ENTITY_DESCRIPTION_SHORT') == "Go-Karts").filter(col('WAIT_TIME_MAX') ==  0)

DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,PARK,USAGE_DATE,attendance
datetime[μs],i64,datetime[μs],str,i64,f64,f64,f64,f64,i64,i64,i64,f64,str,datetime[μs],i64
2018-06-04 09:00:00,9,2018-06-04 09:15:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 09:15:00,9,2018-06-04 09:30:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 09:30:00,9,2018-06-04 09:45:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 09:45:00,9,2018-06-04 10:00:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 10:00:00,10,2018-06-04 10:15:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 10:15:00,10,2018-06-04 10:30:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 10:30:00,10,2018-06-04 10:45:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 10:45:00,10,2018-06-04 11:00:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
2018-06-04 11:00:00,11,2018-06-04 11:15:00,"""Go-Karts""",0,0.0,0.0,0.0,0.0,0,0,0,4.0,"""PortAventura W…",2018-06-04 00:00:00,37617
